In [5]:
import os
import time

In [6]:
#Size of the file
os.path.getsize('Parking_Violations_2017.csv')

2086913576

# Read in the data with Dask

In [7]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('Parking_Violations_2017.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.049002647399902344 sec


# Read in the data with Pandas

In [4]:
import pandas as pd
start = time.time()
df = pd.read_csv('Data/Parking_Violations_2017.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

C:\Users\1998a\AppData\Local\Temp\ipykernel_16408\1646183780.py:3: DtypeWarning: Columns (18,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/Parking_Violations_2017.csv')


Read csv with pandas:  218.9015564918518 sec


# Read in the data with Modin and Ray

In [8]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('Data/Parking_Violations_2017.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2023-08-09 14:41:26,257	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-09 14:41:43,732	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Read csv with modin and ray:  130.88630771636963 sec


# Here Dask is better than Pandas, Modin and Ray, with the least reading time of 0.04 sec

In [10]:
from dask import dataframe as dd
df = dd.read_csv('Parking_Violations_2017.csv',delimiter=',')

In [11]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 43 entries, Summons Number to Double Parking Violation
dtypes: object(23), float64(7), int64(13)

In [12]:
#No. of Rows
len(df)

10803028

In [13]:
#No, of Columns
len(df.columns)

43

In [14]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [15]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [16]:
data=df.columns
data

Index(['SummonsNumber', 'PlateID', 'RegistrationState', 'PlateType',
       'IssueDate', 'ViolationCode', 'VehicleBodyType', 'VehicleMake',
       'IssuingAgency', 'StreetCode1', 'StreetCode2', 'StreetCode3',
       'VehicleExpirationDate', 'ViolationLocation', 'ViolationPrecinct',
       'IssuerPrecinct', 'IssuerCode', 'IssuerCommand', 'IssuerSquad',
       'ViolationTime', 'TimeFirstObserved', 'ViolationCounty',
       'ViolationInFrontOfOrOpposite', 'HouseNumber', 'StreetName',
       'IntersectingStreet', 'DateFirstObserved', 'LawSection', 'SubDivision',
       'ViolationLegalCode', 'DaysParkingInEffect', 'FromHoursInEffect',
       'ToHoursInEffect', 'VehicleColor', 'UnregisteredVehicle?',
       'VehicleYear', 'MeterNumber', 'FeetFromCurb', 'ViolationPostCode',
       'ViolationDescription', 'NoStandingorStoppingViolation',
       'HydrantViolation', 'DoubleParkingViolation'],
      dtype='object')

# Validation

In [41]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import yaml

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [42]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Parking_Violations_2017
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - SummonsNumber
    - PlateID
    - RegistrationState
    - PlateType
    - IssueDate
    - ViolationCode
    - VehicleBodyType
    - VehicleMake
    - IssuingAgency
    - StreetCode1
    - StreetCode2
    - StreetCode3
    - VehicleExpirationDate
    - ViolationLocation
    - ViolationPrecinct
    - IssuerPrecinct
    - IssuerCode
    - IssuerCommand
    - IssuerSquad
    - ViolationTime
    - TimeFirstObserved
    - ViolationCountry
    - ViolationInFrontOfOrOpposite
    - HouseNumber
    - StreetName
    - IntersectingStreet
    - DateFirstObserved
    - LawSection
    - SubDivision
    - ViolationLegalCode
    - DaysParkingInEffect
    - FromHoursInEffect
    - ToHoursInEffect
    - VehicleColor
    - UnregisteredVehicle?
    - VehicleYear
    - MeterNumber
    - FeetFromCurb
    - ViolationPostCode
    - ViolationDescription
    - NoStandingorStoppingViolation
    - HydrantViolation
    - DoubleParkingViolation

Overwriting store.yaml


In [1]:
# Reading config file
import yaml
import utility
config_data = utility.read_config_file("store.yaml")

In [2]:
#data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'Parking_Violations_2017',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['SummonsNumber',
  'PlateID',
  'RegistrationState',
  'PlateType',
  'IssueDate',
  'ViolationCode',
  'VehicleBodyType',
  'VehicleMake',
  'IssuingAgency - StreetCode1 - StreetCode2 - StreetCode3 - VehicleExpirationDate - ViolationLocation - ViolationPrecinct - IssuerPrecinct - IssuerCode - IssuerCommand - IssuerSquad - ViolationTime - TimeFirstObserved - ViolationCountry - ViolationInFrontOfOrOpposite - HouseNumber - StreetName - IntersectingStreet - DateFirstObserved - LawSection - SubDivision - ViolationLegalCode - DaysParkingInEffect - FromHoursInEffect - ToHoursInEffect - VehicleColor - UnregisteredVehicle? - VehicleYear - MeterNumber - FeetFromCurb - ViolationPostCode - ViolationDescription - NoStandingorStoppingViolation - HydrantViolation - DoubleParkingViolation']}

In [17]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv('Data/Parking_Violations_2017.csv',delimiter=',')


In [4]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "Data/" + config_data['file_name'] + f'.{file_type}'

In [23]:
import pandas as pd
df = pd.read_csv(source_file)

In [25]:
#validating the header of the file
utility.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['issuer_code', 'sub_division', 'street_code3', 'feet_from_curb', 'summons_number', 'to_hours_in_effect', 'violation_county', 'days_parking_in_effect', 'violation_precinct', 'hydrant_violation', 'violation_time', 'plate_type', 'issuing_agency', 'date_first_observed', 'unregistered_vehicle', 'double_parking_violation', 'time_first_observed', 'street_code1', 'vehicle_color', 'house_number', 'from_hours_in_effect', 'issuer_command', 'registration_state', 'plate_id', 'street_name', 'issuer_precinct', 'issuer_squad', 'vehicle_expiration_date', 'vehicle_year', 'street_code2', 'violation_description', 'violation_post_code', 'violation_legal_code', 'no_standing_or_stopping_violation', 'violation_in_front_of_or_opposite', 'vehicle_body_type', 'vehicle_make', 'violation_location', 'violation_code', 'law_section', 'issue_date', 'meter_number', 'intersecting_street']
Following YAML columns are not in th

0

In [26]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['summons_number', 'plate_id', 'registration_state', 'plate_type',
       'issue_date', 'violation_code', 'vehicle_body_type', 'vehicle_make',
       'issuing_agency', 'street_code1', 'street_code2', 'street_code3',
       'vehicle_expiration_date', 'violation_location', 'violation_precinct',
       'issuer_precinct', 'issuer_code', 'issuer_command', 'issuer_squad',
       'violation_time', 'time_first_observed', 'violation_county',
       'violation_in_front_of_or_opposite', 'house_number', 'street_name',
       'intersecting_street', 'date_first_observed', 'law_section',
       'sub_division', 'violation_legal_code', 'days_parking_in_effect',
       'from_hours_in_effect', 'to_hours_in_effect', 'vehicle_color',
       'unregistered_vehicle', 'vehicle_year', 'meter_number',
       'feet_from_curb', 'violation_post_code', 'violation_description',
       'no_standing_or_stopping_violation', 'hydrant_violation',
       'double_parking_violation'],
      dtype=

In [28]:
if utility.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['issuer_code', 'sub_division', 'street_code3', 'feet_from_curb', 'summons_number', 'to_hours_in_effect', 'violation_county', 'days_parking_in_effect', 'violation_precinct', 'hydrant_violation', 'violation_time', 'plate_type', 'issuing_agency', 'date_first_observed', 'unregistered_vehicle', 'double_parking_violation', 'time_first_observed', 'street_code1', 'vehicle_color', 'house_number', 'from_hours_in_effect', 'issuer_command', 'registration_state', 'plate_id', 'street_name', 'issuer_precinct', 'issuer_squad', 'vehicle_expiration_date', 'vehicle_year', 'street_code2', 'violation_description', 'violation_post_code', 'violation_legal_code', 'no_standing_or_stopping_violation', 'violation_in_front_of_or_opposite', 'vehicle_body_type', 'vehicle_make', 'violation_location', 'violation_code', 'law_section', 'issue_date', 'meter_number', 'intersecting_street']
Following YAML columns are not in th

In [32]:
import csv

csv_filename = "Parking_Violations_2017.csv"
formatted_data = []

with open(csv_filename, "r") as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        formatted_row = "|".join(row)
        formatted_data.append(formatted_row)

final_data = "\n".join(formatted_data)


In [39]:
import gzip

output_filename = "Parking_Violation_2017.csv.gz/Parking_2017_compressed.gz"
with gzip.open(output_filename, "wb") as f:
    f.write(final_data.encode("utf-8"))


In [46]:
#number of files in gz format folder
import os
entries = os.listdir('Parking_Violation_2017.csv.gz/')
for entry in entries:
    print(entry)

Parking_2017_compressed.gz


In [47]:
#size of the gz format folder
os.path.getsize('Parking_Violation_2017.csv.gz')

0